In [ ]:
중복값을 제거하지 않은 상태에서 진행했습니다.
중복값을 제거한 titanic_first보다 좋은 성능을 보입니다.
0과 1로 나뉘는 이진분류 모델에서는 랜덤 포레스트나 XGBoost모델은 적합하지 않은것 같습니다.


In [2]:
import seaborn as sns
import pandas as pd
import numpy as np

In [3]:
titanic = sns.load_dataset('titanic')

In [4]:
titanic.head(5)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [5]:
titanic.describe()

,survived,pclass,age,sibsp,parch,fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [6]:
titanic.isnull().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [7]:
titanic['age'].fillna(titanic['age'].median(), inplace=True)

titanic['embarked'].fillna(titanic['embarked'].mode()[0], inplace=True)
 
titanic['embark_town'].fillna('Unknown', inplace=True)

titanic['deck'] = titanic['deck'].astype(str)

titanic['deck'].fillna('Unknown', inplace=True)

titanic_no_nan = titanic

C:\Users\USER\AppData\Local\Temp\ipykernel_19992\3133005395.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  titanic['age'].fillna(titanic['age'].median(), inplace=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_19992\3133005395.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy

In [8]:
titanic_no_nan['sex'] = titanic_no_nan['sex'].map(lambda x: 0 if x == 'male' else 1 )

titanic_no_nan['alive'] = titanic_no_nan['alive'].map(lambda x: 0 if x == 'yes' else 1)

titanic_no_nan['embarked'] = titanic_no_nan['embarked'].map(lambda x: 0 if x == 'C' else 1 if x == 'Q' else 2)

titanic_no_nan['class'] = titanic_no_nan['class'].map(lambda x: 0 if x =='First' else 1 if x =='Second' else 2) 

titanic_no_nan['who'] = titanic_no_nan['who'].map(lambda x: 0 if x == 'man' else 1)

titanic_no_nan['adult_male'] = titanic_no_nan['adult_male'].map(lambda x: 0 if x == 'False' else 1)

titanic_no_nan['deck'] = titanic_no_nan['deck'].map(lambda x:  0 if x == 'A' else 1 if x == 'B' else 2 if x == 'C' else 3 if x == 'D' else 4 if x == 'E' else 5 if x == 'F' else 6)

titanic_no_nan['embark_town'] = titanic_no_nan['embark_town'].map(lambda x: 1 if x == 'Cherbourg' else 2 if x == 'Queenstown' else 3 if x == 'Southampton' else 0)

titanic_no_nan['alone'] = titanic_no_nan['alone'].map(lambda x: 0 if x == 'False' else 1)

titanic_no_nan['class'].astype(str)

titanic_enco = titanic_no_nan 

In [11]:
titanic_enco['family_size'] = titanic_enco['sibsp'] + titanic_enco['parch'] + 1

In [13]:
#로지스틱 모델

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

data = titanic_enco

data = data[['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked', 'family_size']].dropna()

X = data.drop('survived', axis=1)
y = data['survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 스케일링
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 모델 생성 및 학습
model = LogisticRegression()
model.fit(X_train, y_train)

# 예측
# model.predict(X_test) 학습된 모델을 사용하여 테스트데이터 X_test에 대한 예측값을 계산.예측 결과는 y_pred에 저장
y_pred = model.predict(X_test)

print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Classification Report:\n{classification_report(y_test, y_pred)}")
print(f"Confusion Matrix:\n{confusion_matrix(y_test, y_pred)}")

# 0.8로 중복값 제거하지 않은 상태에서 더 좋은 성능을 보인다.
# Classification의 크기 차이가 더 작아진 것을 확인(학습이 더 잘 진행 되었다.)

Accuracy: 0.8044692737430168
Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.86      0.84       105
           1       0.78      0.73      0.76        74

    accuracy                           0.80       179
   macro avg       0.80      0.79      0.80       179
weighted avg       0.80      0.80      0.80       179

Confusion Matrix:
[[90 15]
 [20 54]]


In [21]:
#랜덤 포레스트 모델

from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

data = titanic_enco

data = data[['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked', 'family_size']].dropna()

X = data.drop('survived', axis=1)
y = data['survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
# 모델 학습
rf_model.fit(X_train_scaled, y_train)

# 예측
y_pred_rf = rf_model.predict(X_test_scaled)

# 평가
mse_rf = mean_squared_error(y_test, y_pred_rf)
print(f'랜덤 포레스트 모델의 MSE: {mse_rf}')

# 훈련 데이터에서 예측
y_train_pred = xgb_model.predict(X_train_scaled)

# 훈련 데이터의 MSE 계산
mse_train = mean_squared_error(y_train, y_train_pred)
print(f'훈련 데이터에 대한 MSE: {mse_train}')

# 두 점수의 차이가 0.04로 전보다 좋아짐
# 이진 분류 문제에서는 예측값이 0 또는 1로 제한되므로 
# MSE가 0.2라는 것은 예측이 완벽하지 않다는 것을 의미

랜덤 포레스트 모델의 MSE: 0.13605153814540247
훈련 데이터에 대한 MSE: 0.0895534323725672


In [20]:
#XGBoost 모델

import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.metrics import mean_squared_error

data = titanic_enco

data = data[['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked', 'family_size']].dropna()

X = data.drop('survived', axis=1)
y = data['survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

xgb_model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# 모델 학습
xgb_model.fit(X_train_scaled, y_train)

# 예측
y_pred_xgb = xgb_model.predict(X_test_scaled)

# 평가
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
print(f'XGBoost 모델의 MSE: {mse_xgb}')

# 훈련 데이터에서 예측
y_train_pred = xgb_model.predict(X_train_scaled)

# 훈련 데이터의 MSE 계산
mse_train = mean_squared_error(y_train, y_train_pred)
print(f'훈련 데이터에 대한 MSE: {mse_train}')

# 두 점수의 차이가 0.04로 전보다 좋아짐
# 이진 분류 문제에서는 예측값이 0 또는 1로 제한되므로 
# MSE가 0.2라는 것은 예측이 완벽하지 않다는 것을 의미

XGBoost 모델의 MSE: 0.12981004899201257
훈련 데이터에 대한 MSE: 0.0895534323725672
